#### Ai Agents in LangGraph Course

In [1]:
import os
import re
from pathlib import Path
from dotenv import load_dotenv

from langchain_groq import ChatGroq

# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

In [29]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated # to construct the agent's state
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver

In [30]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add] # operator.add allows us to add messages instead of replacing them when the LLM's output is returned

In [4]:
# # Used for persistence
# from langgraph.checkpoint.sqlite import SqliteSaver
# memory = SqliteSaver.from_conn_string(":memory:")

In [37]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState) # initialize graph
        memory = MemorySaver()

        # Add nodes
        graph.add_node("llm", self.call_groq)
        graph.add_node("action", self.take_action)
        
        # The edge where the decision to use a tool is made
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        
        # Create edge and set starting point of the graph
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        self.graph = graph.compile(checkpointer=memory) # Build graph
        
        self.tools = {t.name: t for t in tools} # Save the tools' names that can be used
        self.model = model.bind_tools(tools) # Provide the name of the tools to the agent

    # Tells the agent if action is needed by checking the last message in the state which is supposed to contain this info
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    # Call the LLM and update the Agent's State by adding the response
    def call_groq(self, state: AgentState):
        messages = state['messages']
        if self.system: messages = [SystemMessage(content=self.system)] + messages
        
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # Search for the tool and use it
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check if tool name is found in list of tools 
                print("\n ....tool name not found in list of tools....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args']) 

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

        print("Back to the model!")
        return {'messages': results}

In [34]:
from langchain.graphs import Neo4jGraph
from langchain_core.tools import tool

# Connect to graph
graph = Neo4jGraph()

@tool
def query_graph(query):
  """Requires get_graph_schema to be run before using this function. This function is to Query from Neo4j knowledge graph using Cypher."""
  return graph.query(query)

In [35]:
task = "Task: You are a career guide. Your job is to ask me up to 15 questions to uncover my personality traits according to the RAISEC model. You will ask these questions in a conversational flow where you will ask the second question after I answer the first. Once you understand my personality, you will stop asking questions and use a Neo4j database to improve your knowledge on compatible career paths for me. You will query the possible occupation titles that are suitable for my character. At any point, I can ask you questions and you will answer normally, then you will continue your personality test."

goal = "Understand my personality and then suggest suitable career paths. Note: when asking your questions, please number them to keep track of the number of questions asked."

schema_context = f"Here is the graph's schema: {graph.structured_schema}."

property_values = f"Property Values: empty"

query_approach = "Querying approach: You will not use 'LIMIT'. If Property Values: empty, you will not use general queries and will not include 'WHERE' or try to specify property values inside your Cypher code."

output = "Your final output: Interpret all the queried data, choose up to 15 suitable careers for me, list them in bullet points and include a brief explanation of how each path suites my personality. Include Cypher code in your answer."

tone = "Output's tone: Make your output friendly, fun and easy to read."

personal_info = "Personal Info: I love people and I am a good listener. I enjoy observation and analysis. I prefer being with abults rather than with kids and I also have computer programming skills."

reminder = "Reminder: If Property Values: empty, you will not use 'WHERE' or try to specify property values inside your Cypher code. Under no circumstances should you use 'DELETE'. Find the occupations that suite my character."

prompt = f"{task}\ {goal}\ {schema_context}\ {property_values}\ {query_approach}\ {output}\ {tone}\ {personal_info}\ {reminder}"

In [39]:
# Prepare agent and the initial prompt
model = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")
abot = Agent(model, [query_graph], system=prompt)

# Prepare Human message
query = "What will we do today"
# messages = [HumanMessage(content=query)]

# # Give the agent the messsage without the Memory in compile()
# result = abot.graph.invoke({"messages": messages})
# print(result['messages'][-1].content)

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content=query)
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What will we do today
================================== Ai Message ==================================

We're going to have some fun today. We'll play a game of self-discovery, and I'll ask you a series of questions to help me understand your personality traits according to the RAISEC model. Don't worry, it's not a test, and there are no right or wrong answers. I just want to get to know you better.

Here's your first question:

1. When working on a project, what motivates you more: the opportunity to be creative and try new things, or the chance to work with others and build strong relationships?


In [40]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I enjoy a balance of both creative work and teamwork.")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I enjoy a balance of both creative work and teamwork.
================================== Ai Message ==================================

It sounds like you're a people person who also values independence and creativity. That's a great combination.

Here's your next question:

2. When faced with a difficult decision, what do you usually rely on to make a choice: logic and analysis, or your intuition and instincts?


In [41]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="A combination of both is necessary for me. But for sure logic and reason dominates.")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

A combination of both is necessary for me. But for sure logic and reason dominates.
================================== Ai Message ==================================

You're a logical thinker who also considers your intuition, but tends to lean more on facts and data to make decisions. That's a very analytical approach.

Here's your next question:

3. In social situations, how would you describe yourself: the life of the party, always telling jokes and trying to make others laugh, or more of a listener, observing and absorbing what's going on around you?


In [42]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I'm more of an observer. However with my peeps I am part of the lide of the group.")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm more of an observer. However with my peeps I am part of the lide of the group.
================================== Ai Message ==================================

It sounds like you're a bit of an introverted extrovert. You enjoy observing and listening, but when you're with people you're close to, you can be more outgoing and playful. That's a great way to balance your energy and connect with others.

Here's your next question:

4. When it comes to planning and organization, do you like to have a clear plan and schedule, or are you more flexible and adaptable, going with the flow?


In [43]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I prefer a clear schedule")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I prefer a clear schedule
================================== Ai Message ==================================

You like to have a plan in place and know what to expect. That's a very organized and structured approach. Having a clear schedule can help you feel more in control and focused.

Here's your next question:

5. How do you feel about repetitive tasks or routine work: do you find it comforting and enjoyable, or do you get bored and restless, craving something new and challenging?


In [44]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I crave new challenges")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I crave new challenges
================================== Ai Message ==================================

You're someone who thrives on novelty and variety. You get bored with routine tasks and crave new challenges that stimulate your mind and keep you engaged. That's a great trait for personal and professional growth.

Here's your next question:

6. When working on a team project, how do you handle conflicts or disagreements with others: do you tend to assert yourself and try to convince others of your point of view, or do you try to find a compromise and mediate the discussion?

By the way, we're halfway through the questions. You're doing great!


In [45]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I communicate my point of view and discuss the suitable approach necessary.")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I communicate my point of view and discuss the suitable approach necessary.
================================== Ai Message ==================================

It sounds like you're a collaborative communicator who values open discussion and finding a mutually beneficial solution. You're not afraid to share your thoughts and opinions, but you also listen to others and try to find a compromise. That's a great way to build strong relationships and achieve common goals.

Here's your next question:

7. How do you feel about working with technology: do you enjoy learning new software and tools, or do you prefer to focus on more people-oriented tasks?

Also, I noticed you mentioned earlier that you have computer programming skills. That's a great asset!


In [46]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="A balance between both, although I prefer human behavior analysis, observation, and commincation")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

A balance between both, although I prefer human behavior analysis, observation, and commincation
================================== Ai Message ==================================

You enjoy working with technology, but your true passion lies in understanding human behavior, observing social dynamics, and communicating with others. You're a people person who also appreciates the value of technology in facilitating your work.

Here's your next question:

8. When it comes to helping others, what motivates you more: the opportunity to provide emotional support and guidance, or the chance to offer practical solutions and advice?

By the way, I think I'm starting to get a good sense of your personality traits. We're getting close to the end of the questions!


In [47]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I prefer helping the person figuring their route. I don't mind giving my input as support")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I prefer helping the person figuring their route. I don't mind giving my input as support
================================== Ai Message ==================================

You're a supportive guide who empowers others to find their own way, rather than telling them what to do. You're happy to offer your input and advice, but ultimately, you want the person to take ownership of their decisions and actions. That's a very empowering approach to helping others.

Here's your next question:

9. How do you feel about working in a fast-paced, dynamic environment: do you thrive on the energy and excitement, or do you prefer a more stable and predictable work setting?

By the way, I think we're almost done with the questions. Just a few more to go!

Also, I'll start preparing the query to find suitable career paths for you based on your answers.


In [48]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I prefer a challenging and growing environment")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I prefer a challenging and growing environment
================================== Ai Message ==================================

You're someone who thrives in environments that challenge you to grow and learn. You're not afraid of uncertainty and are motivated by the opportunity to develop new skills and take on new responsibilities. That's a great attitude for personal and professional growth.

I think that's all the questions I need to ask you. You've given me a great sense of your personality traits and preferences.

Now, I'll use the RAISEC model to analyze your answers and identify some suitable career paths for you. I'll also query the Neo4j database to find occupations that match your personality traits.

Here's the Cypher query I'll use:
```
MATCH (o:Occupation)-[:need_for_personality_trait]->(t:Personality_Trait)
RETURN o.title AS Occupation, COLLECT(t.title) AS Personality_Traits
```
This query 

In [49]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Okay")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Okay
================================== Ai Message ==================================
Tool Calls:
  query_graph (call_zeyz)
 Call ID: call_zeyz
  Args:
    query: MATCH (o:Occupation)-[:need_for_personality_trait]->(t:Personality_Trait) RETURN o.title AS Occupation, COLLECT(t.title) AS Personality_Traits
Calling: {'name': 'query_graph', 'args': {'query': 'MATCH (o:Occupation)-[:need_for_personality_trait]->(t:Personality_Trait) RETURN o.title AS Occupation, COLLECT(t.title) AS Personality_Traits'}, 'id': 'call_zeyz', 'type': 'tool_call'}
Back to the model!
================================= Tool Message =================================
Name: query_graph

[{'Occupation': 'School_Psychologists', 'Personality_Traits': ['Conventional', 'Social', 'Investigative']}, {'Occupation': 'Film_and_Video_Editors', 'Personality_Traits': ['Conventional', 'Artistic']}, {'Occupation': 'Editors', 'Personality_Traits': ['Con

In [50]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Why didn't you suggest Psychologist roles?")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Why didn't you suggest Psychologist roles?
================================== Ai Message ==================================

Based on your answers, I did consider Psychologist roles as a potential fit for you. However, I didn't include them in my initial list of suggestions for a few reasons:

1. **Competition**: Psychologist roles often require advanced degrees (e.g., Ph.D. or Psy.D.) and can be highly competitive. I wanted to suggest roles that might be more accessible with your current skills and experience.
2. **Specialization**: Psychologist roles often require specialization in a particular area, such as clinical psychology, counseling psychology, or neuropsychology. I didn't have enough information to determine which area might be the best fit for you.
3. **Licensure**: Psychologist roles often require licensure, which can vary by state or country. I didn't want to suggest a role that might require

In [32]:
# ## Visualize Graph
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

from IPython.display import Image, display

try:
    display(Image(abot.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    print("Nono")
    pass

Nono
